In [8]:
#from build_predictions import * 
#from GaussRank import GaussRankMap
import pandas as pd 
#from create_parquet import *
#from data import *
import warnings
warnings.filterwarnings("ignore") 

In [9]:
DATA_DIR='/rapids/notebooks/srabhi/champs-2019/input/'

##### Get the three frames needed for building train / validation oupling stacking 

## Nodes frame

In [10]:
node_frame = pd.read_csv(DATA_DIR+'parquet/baseline_node_frame.csv')

In [11]:
node_frame.head(2)

,symbol,acceptor,donor,aromatic,hybridization,num_h,atomic,num_nodes,node_dim,molecule_name,atom_index
0,2.0,0.0,0.0,0.0,3.0,3.0,6.0,21,7,dsgdb9nsd_101594,0
1,3.0,0.0,1.0,0.0,2.0,1.0,7.0,21,7,dsgdb9nsd_101594,1


- Molecule node frame 

In [5]:
node_cols = ['symbol','acceptor', 'donor', 'aromatic',  'hybridization', 'num_h', 'atomic']
shared_cols = ['molecule_name', 'num_nodes', 'node_dim']
tmp = node_frame.groupby('molecule_name').apply(lambda x: x[node_cols].values.reshape(-1))
molecule_node = pd.DataFrame(tmp.values.tolist())
#pad node max 29 to 32 
pad_cols = 21
d = dict.fromkeys([str(i) for i in range(molecule_node.shape[1], molecule_node.shape[1]+pad_cols)], 0.0)
molecule_node = molecule_node.assign(**d).fillna(0.0)
molecule_node['molecule_name'] = tmp.index
molecule_node = molecule_node.merge(node_frame[shared_cols].drop_duplicates(), on='molecule_name', how='left')
cols = molecule_node.columns.tolist()
new_cols = cols[-3:] + cols[:-3]
molecule_node = molecule_node[new_cols]
molecule_node.columns = ['molecule_name', 'num_nodes', 'node_dim'] + ['node_%s'%i for i in range(NODE_MAX*7)]
molecule_node.head(2)

,molecule_name,num_nodes,node_dim,node_0,node_1,node_2,node_3,node_4,node_5,node_6,...,node_214,node_215,node_216,node_217,node_218,node_219,node_220,node_221,node_222,node_223
0,dsgdb9nsd_000001,5,7,2.0,0.0,0.0,0.0,3.0,4.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,dsgdb9nsd_000002,4,7,3.0,0.0,1.0,0.0,3.0,3.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Edges frame

In [6]:
edge_frame = pd.read_csv(DATA_DIR+'parquet/baseline_edge_frame.csv')

In [7]:
edge_frame.head(2)

,atom_index_0,atom_index_1,edge_type,distance,angle,molecule_name,num_edge,edge_dim
0,0.0,1.0,1.0,1.450358,-0.498035,dsgdb9nsd_101594,420,5
1,0.0,2.0,0.0,2.454559,-0.499508,dsgdb9nsd_101594,420,5


- Molecule edge frame 

In [8]:
edge_cols = ['atom_index_0', 'atom_index_1', 'edge_type', 'distance', 'angle' ]
shared_cols = ['molecule_name', 'num_edge', 'edge_dim']
tmp = edge_frame.groupby('molecule_name').apply(lambda x: x[edge_cols].values.reshape(-1))
molecule_edge = pd.DataFrame(tmp.values.tolist())
#pad edge_max 812 to 816
pad_cols = 4 * 5
d = dict.fromkeys([str(i) for i in range(molecule_edge.shape[1], molecule_edge.shape[1]+pad_cols)], 0.0)
molecule_edge = molecule_edge.assign(**d).fillna(0.0)
molecule_edge['molecule_name'] = tmp.index
molecule_edge = molecule_edge.merge(edge_frame[shared_cols].drop_duplicates(), on='molecule_name', how='left')
cols = molecule_edge.columns.tolist()
new_cols = cols[-3:] + cols[:-3]
molecule_edge = molecule_edge[new_cols]
molecule_edge.columns = ['molecule_name', 'num_edge', 'edge_dim']+ ['edge_%s'%i for i in range(EDGE_MAX*5)]
molecule_edge.head(2)

,molecule_name,num_edge,edge_dim,edge_0,edge_1,edge_2,edge_3,edge_4,edge_5,edge_6,...,edge_4070,edge_4071,edge_4072,edge_4073,edge_4074,edge_4075,edge_4076,edge_4077,edge_4078,edge_4079
0,dsgdb9nsd_000001,20,5,0.0,1.0,1.0,1.091953,-0.901529,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,dsgdb9nsd_000002,12,5,0.0,1.0,1.0,1.017190,0.292853,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Coupling frame**

In [9]:
coupling_frame = pd.read_csv(DATA_DIR+'parquet/baseline_coupling_frame.csv')

In [10]:
coupling_frame.head(2)

,atom_index_0,atom_index_1,coupling_type,scalar_coupling,fc,sd,pso,dso,id,num_coupling,coupling_dim,molecule_name
0,9.0,0.0,0.0,84.50560,83.117900,0.133123,0.548422,0.706185,3528934.0,64,9,dsgdb9nsd_101594
1,9.0,1.0,4.0,-0.42038,-0.454646,-0.012691,0.098126,-0.051169,3528935.0,64,9,dsgdb9nsd_101594


# Build Train / validation gaussrank value for each fold 

In [20]:
from time import time
def save_cv_data(fold, coupling_frame):
    print('fold: %s' %fold)
    split_train = 'train_split_by_mol_hash.%s.npy'%fold
    split_valid = 'valid_split_by_mol_hash.%s.npy'%fold
    id_train_ = np.load(DATA_DIR + '/split/%s'%split_train,allow_pickle=True)
    id_valid_ = np.load(DATA_DIR + '/split/%s'%split_valid,allow_pickle=True)
    csv = 'test'
    df = pd.read_csv(DATA_DIR + '/csv/%s.csv'%csv)
    id_test_ = df.molecule_name.unique()
    
    train = coupling_frame[coupling_frame.molecule_name.isin(id_train_)]
    validation = coupling_frame[coupling_frame.molecule_name.isin(id_valid_)]
    test = coupling_frame[coupling_frame.molecule_name.isin(id_test_)]
    
    # Get GaussRank of coupling values 
    t0 = time()
    grm = GaussRankMap()
    df_train = train[['coupling_type', 'scalar_coupling']]
    df_valid = validation[['coupling_type', 'scalar_coupling']]
    df_train.columns = ['type', 'scalar_coupling_constant']
    df_valid.columns = ['type', 'scalar_coupling_constant']
    # Reverse type mapping 
    df_train.type = df_train.type.map(REVERSE_COUPLING_TYPE)
    df_valid.type = df_valid.type.map(REVERSE_COUPLING_TYPE)
    #fit grm 
    transformed_training = grm.fit_training(df_train, reset=True)
    transformed_validation = grm.convert_df(df_valid, from_coupling=True)
    validation['gaussrank_coupling'] =  transformed_validation
    train['gaussrank_coupling'] = transformed_training
    print('Getting gaussrank transformation for train/validation data took %s seconds' %(time()-t0))
    print(grm.coupling_order)
    test['gaussrank_coupling'] = 0 

    general_coupling_frame = pd.concat([train, validation, test])

    # Build molecule coupling frame for fold 
    coupling_cols = ['atom_index_0', 'atom_index_1', 'coupling_type', 'scalar_coupling', 'gaussrank_coupling',  'fc', 'sd', 'pso', 'dso', 'id']
    shared_cols = ['molecule_name', 'num_coupling', 'coupling_dim']

    tmp = general_coupling_frame.groupby('molecule_name').apply(lambda x: x[coupling_cols].values.reshape(-1))
    molecule_coupling = pd.DataFrame(tmp.values.tolist())
    # pad coupling_max from 135 to 136
    pad_cols = 10
    d = dict.fromkeys([str(i) for i in range(molecule_coupling.shape[1], molecule_coupling.shape[1]+pad_cols)], 0.0)
    molecule_coupling = molecule_coupling.assign(**d).fillna(0.0)
    molecule_coupling['molecule_name'] = tmp.index
    molecule_coupling = molecule_coupling.merge(general_coupling_frame[shared_cols].drop_duplicates(), on='molecule_name', how='left')

    cols = molecule_coupling.columns.tolist()
    new_cols = cols[-3:] + cols[:-3]
    molecule_coupling = molecule_coupling[new_cols]
    molecule_coupling.columns = ['molecule_name', 'num_coupling', 'coupling_dim'] + ['coupling_%s'%i for i in range(COUPLING_MAX*10)]

    print(molecule_coupling.shape, molecule_edge.shape, molecule_node.shape)

    node_edge_frame = pd.merge(molecule_node, molecule_edge, on='molecule_name', how='left')
    general_stack_frame =  pd.merge(node_edge_frame, molecule_coupling, on='molecule_name', how='left')

    train_frame = general_stack_frame[general_stack_frame.molecule_name.isin(id_train_)]
    validation_frame = general_stack_frame[general_stack_frame.molecule_name.isin(id_valid_)]
    test_frame = general_stack_frame[general_stack_frame.molecule_name.isin(id_test_)]

    validation_frame.to_parquet(DATA_DIR+'/parquet/fold_%s'%fold+'/validation.parquet')
    train_frame.to_parquet(DATA_DIR +'/parquet/fold_%s'%fold+ '/train.parquet')
    # save mapping
    for i, (type_, frame) in enumerate(zip(grm.coupling_order, grm.training_maps)): 
        frame.to_csv(DATA_DIR +'/parquet/fold_%s'%fold+'/mapping_type_%s_order_%s.csv'%(type_, i), index=False)
    return test_frame

In [21]:
test_frame = save_cv_data(1, coupling_frame) 

fold: 1
Getting gaussrank transformation for train/validation data took 7.560222148895264 seconds
['1JHC', '2JHC', '3JHC', '2JHH', '3JHH', '1JHN', '3JHN', '2JHN']
(130775, 1363) (130775, 4083) (130775, 227)


In [ ]:
%%time
folds = 4
for fold in range(1, folds):
    print(coupling_frame.shape)
    test_frame = save_cv_data(fold, coupling_frame)  

In [23]:
test_frame.to_parquet(DATA_DIR +'/parquet/test.parquet')